# Electron equation of motion due to Lorentz force and [Ford&O'Connell1991](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.524.6982&rep=rep1&type=pdf) form of the Abraham-Lorentz force

\begin{align*}
m_e \frac{d \vec{v}}{d t} &= \vec{F}_{Lorentz} + \frac{q^2}{6 \pi \epsilon_0 c^3 m_e}\left(\frac{d \vec{F}_{Lorentz}}{dt}\right)\\
&= \vec{F}_{Lorentz} + \tau\left(\frac{d \vec{F}_{Lorentz}}{dt}\right)
\end{align*}

<div class="alert alert-block alert-danger">
<b>Note:</b> Above equation is in SI. Ford and O'Connell is in CGS.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
plt.rcParams['figure.figsize'] = [4.5, 3]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['lines.dashed_pattern'] = [3.5,3.5]

In [ ]:
# Problem parameters. Not real values
B = 1.5
m = 3
q = -1

# Note, this is a signed value
omega = B * q / m

tau = 0.025 # Not the real value. Chosen to test scheme

In [ ]:
# A very basic error plotting routine
def error_plot(x, y, x_exact=None, y_exact=None, title=None, xlabel=None, ylabel=None):
    plt.plot(x, y)
    
    if y_exact is not None:
        if x_exact is not None:
            plt.plot(x_exact, y_exact, linestyle='--')
        else:
            plt.plot(x, y_exact, linestyle='--')
            
    plt.title(title, fontsize=14)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    
    if np.all(x >= 0.0):
        plt.xlim(left=0.0)
    
    plt.tight_layout()

## Rewrite RHS of Ford 1991 from:

\begin{align*}
m \frac{dv_x}{dt} &= q v_y B_z + q B_z \tau \frac{d}{dt}v_y \\
m \frac{dv_y}{dt} &= - q v_x B_z - q B_z \tau \frac{d}{dt}v_x \\
\end{align*}

## To:

\begin{align*}
\frac{d v_x}{dt} &= \left(\omega v_y - \omega^2 \tau v_x\right)\left(\frac{1}{1 + \omega^2 \tau^2}\right)\\
\frac{d v_y}{dt} &= \left(-\omega v_x - \omega^2 \tau v_y\right)\left(\frac{1}{1 + \omega^2 \tau^2}\right)
\end{align*}

Where $\omega = \frac{qB_z}{m}$ is the ***signed*** angular frequency.

In [ ]:
# RHS according to Ford & O'Connell (1991). Non-relativistic
def rhs_ford1991(t, x, q, m, B, tau):
    # Note, this is a signed value
    omega = B * q / m
    
    ax = (omega * x[3] - tau * omega**2 * x[2]) / (1 + tau**2 * omega**2)
    ay = (-omega * x[2] - tau * omega**2 * x[3]) / (1 + tau**2 * omega**2)
    return [x[2], x[3], ax, ay]

## Writing $\mu = \tau \omega^2$, the velocity solution is:

\begin{align*}
v_x &= \frac{e^{-\mu t} \sin(\omega t)}{1 + \tau^2 \omega^2} \\
v_y &= \frac{e^{-\mu t} \cos(\omega t)}{1 + \tau^2 \omega^2}
\end{align*}

## Integrate to obtain position solutions:

\begin{align*}
x &= \frac{-e^{-\mu t} \left(\mu\sin(\omega t) + \omega \cos(\omega t)\right)}{\left(\omega^2 + \mu^2\right)\left(1 + \tau^2 \omega^2\right)} \\
y &= \frac{e^{-\mu t} \left(\omega\sin(\omega t) - \mu \cos(\omega t)\right)}{\left(\omega^2 + \mu^2\right)\left(1 + \tau^2 \omega^2\right)}
\end{align*}

In [ ]:
# Analytic solution. Assumes motion initially vertical
def analytic_solution(t, q, m, B, tau):
    omega = q * B / m
    mu = tau * omega**2
    
    vx_soln = (np.exp(-mu * t) * np.sin(omega*t)) / (1.0 + tau**2 * omega**2)
    vy_soln = (np.exp(-mu * t) * np.cos(omega*t)) / (1.0 + tau**2 * omega**2)

    x_soln = -np.exp(-mu * t) * (mu * np.sin(omega * t)
                                   + omega * np.cos(omega * t)) / ((omega**2 + mu**2)
                                                                   * (1 + tau**2 * omega**2))
    y_soln = np.exp(-mu * t) * (omega * np.sin(omega * t)
                                   - mu * np.cos(omega * t)) / ((omega**2 + mu**2)
                                                                * (1 + tau**2 * omega**2))
    return x_soln, y_soln, vx_soln, vy_soln

In [ ]:
# Solve Ford&O'Connell 1991 for n rotations
def solve_ford1991(n, q, m, B, tau):
    # Maximum timestep. Could probably be smaller
    max_step = 1e-3 / np.abs(omega)

    # Set initial conditions
    x0, y0, vx0, vy0 = analytic_solution(0, q, m, B, tau)
    # Note that for tau /= 0, both x0 and y0 and non-zero
    ic = [x0, y0, vx0, vy0]

    res = solve_ivp(rhs_ford1991, (0,n * 2.0 * np.pi / np.abs(omega)), ic, max_step=max_step, args=[q, m, B, tau])
    
    return res

In [ ]:
res = solve_ford1991(5, q, m, B, tau)

In [ ]:
# Get analytic solution
x_soln, y_soln, vx_soln, vy_soln = analytic_solution(res.t, q, m, B, tau)

In [ ]:
error_plot(res.y[0], res.y[1], x_exact=x_soln, y_exact=y_soln,
          title='Electron Trajectory', xlabel='x', ylabel='y')

In [ ]:
ke = 0.5 * m * (res.y[2]**2 + res.y[3]**2)
mu = omega**2 * tau
taue = - 2.0 * mu / (1.0 + tau**2)
error_plot(res.t, ke, y_exact=ke[0]*np.exp(taue * res.t), title='Energy decay',
          xlabel='t', ylabel='Electron energy')

In [ ]:
error_plot(res.t, res.y[2], y_exact=vx_soln, xlabel='t', ylabel=r'$v_x$')

In [ ]:
error_plot(res.t, res.y[3], y_exact=vy_soln, xlabel='t', ylabel=r'$v_y$')

In [ ]:
error_plot(res.t, res.y[0], y_exact=x_soln, xlabel='t', ylabel=r'$x$')

In [ ]:
error_plot(res.t, res.y[1], y_exact=y_soln, xlabel='t', ylabel=r'$y$')

In [ ]:
rad = np.sqrt(res.y[1]**2 + res.y[0]**2)
rad_exact = np.sqrt(y_soln**2 + x_soln**2)
error_plot(res.t, rad, y_exact=rad_exact, xlabel='t', ylabel=r'radius')

In [ ]:
# Error in x,y as a function of time
plt.plot(res.t, res.y[0] - x_soln)
plt.plot(res.t, res.y[1] - y_soln)

In [ ]:
# Error in v_x, v_y as a function of time
plt.plot(res.t, res.y[2] - vx_soln)
plt.plot(res.t, res.y[3] - vy_soln)